In [1]:
with open("input.txt", "r") as f:
    logs = [l.strip() for l in f]

In [2]:
logs[:10]

['[1518-10-03 00:47] falls asleep',
 '[1518-07-26 23:50] Guard #487 begins shift',
 '[1518-06-22 00:48] wakes up',
 '[1518-08-21 00:30] falls asleep',
 '[1518-11-21 00:55] wakes up',
 '[1518-05-30 00:06] falls asleep',
 '[1518-04-09 00:44] wakes up',
 '[1518-07-22 00:58] wakes up',
 '[1518-06-15 00:57] wakes up',
 '[1518-09-13 00:31] wakes up']

In [3]:
# Parse the damn logs and randomly check
import pandas as pd
import numpy as np

months = [l.split('-')[1] for l in logs]
days = [l.split('-')[2].split(' ')[0] for l in logs]
hours = [l.split(':')[0].split(' ')[1] for l in logs]
minutes = [l.split(':')[1].split(']')[0] for l in logs]

for i in np.random.choice(range(len(logs)), 5, replace=False):
    print logs[i].rsplit(' ', 1)[0]
    params = []
    for param in [months, days, hours, minutes]:
        params.append(param[i])
    print params 

[1518-09-29 00:55] wakes
['09', '29', '00', '55']
[1518-09-13 00:35] falls
['09', '13', '00', '35']
[1518-03-24 00:55] wakes
['03', '24', '00', '55']
[1518-08-29 23:50] Guard #2179 begins
['08', '29', '23', '50']
[1518-09-08 23:56] Guard #2833 begins
['09', '08', '23', '56']


In [4]:
dates = [l.split(' ')[0].split('[')[1] for l in logs]
times = [l.split(' ')[1].split(']')[0] for l in logs]
datetimes = [d+' '+t for (d,t) in zip(dates, times)]
events = [l.split(' ',2)[-1] for l in logs]

for i in np.random.choice(range(len(logs)), 5, replace=False):
    print logs[i]
    params = []
    for param in [dates, times, events]:
        params.append(param[i])
    print datetimes[i]
    print params 
    print ''

[1518-08-07 23:47] Guard #659 begins shift
1518-08-07 23:47
['1518-08-07', '23:47', 'Guard #659 begins shift']

[1518-04-19 00:04] Guard #2917 begins shift
1518-04-19 00:04
['1518-04-19', '00:04', 'Guard #2917 begins shift']

[1518-10-20 00:56] wakes up
1518-10-20 00:56
['1518-10-20', '00:56', 'wakes up']

[1518-03-20 00:03] Guard #1811 begins shift
1518-03-20 00:03
['1518-03-20', '00:03', 'Guard #1811 begins shift']

[1518-05-14 00:01] falls asleep
1518-05-14 00:01
['1518-05-14', '00:01', 'falls asleep']



In [16]:
guard_logs = pd.DataFrame({'Dates': dates,
                           'Times': times,
                           'Datetime': datetimes,
                           'Events': events})

In [17]:
sorted_logs = guard_logs.sort_values(by='Datetime')
sorted_logs.head()

,Dates,Datetime,Events,Times
388,1518-03-04,1518-03-04 00:00,Guard #659 begins shift,00:00
460,1518-03-04,1518-03-04 00:39,falls asleep,00:39
463,1518-03-04,1518-03-04 00:47,wakes up,00:47
85,1518-03-05,1518-03-05 00:04,Guard #2161 begins shift,00:04
226,1518-03-05,1518-03-05 00:22,falls asleep,00:22


In [18]:
# Add guard id
guard_ids = []
shift_num = 0
shift_ids = []
guard_id = None
for day in sorted_logs.Dates.unique():
    for entry in sorted_logs.loc[sorted_logs.Dates == day].Events:
        if '#' in entry:
            guard_id = entry.split('#')[1].split(' ')[0]
            shift_num += 1
        guard_ids.append(guard_id)
        shift_ids.append(shift_num)
        

In [19]:
sorted_logs['Guard_id'] = np.array(guard_ids)
sorted_logs['Shift_id'] = np.array(shift_ids)
sorted_logs.head()

,Dates,Datetime,Events,Times,Guard_id,Shift_id
388,1518-03-04,1518-03-04 00:00,Guard #659 begins shift,00:00,659,1
460,1518-03-04,1518-03-04 00:39,falls asleep,00:39,659,1
463,1518-03-04,1518-03-04 00:47,wakes up,00:47,659,1
85,1518-03-05,1518-03-05 00:04,Guard #2161 begins shift,00:04,2161,2
226,1518-03-05,1518-03-05 00:22,falls asleep,00:22,2161,2


In [20]:
unique_guard_ids = sorted_logs['Guard_id'].unique()
number_guards = len(unique_guard_ids)

guard_index_dict = {}
guard_id_dict = {}

for idx in range(number_guards):
    guard_index_dict[unique_guard_ids[idx]] = idx
    guard_id_dict[idx] = unique_guard_ids[idx]
    
time_range = 60*2

timesheet = np.zeros((number_guards, time_range))

In [23]:
timesheet = np.zeros((number_guards, time_range))
for shift in sorted_logs.Shift_id.unique():
    start_sleep = None
    guard_id = sorted_logs.loc[sorted_logs.Shift_id == shift].iloc[0].Guard_id
    timesheet_idx = guard_index_dict[guard_id]
    for idx, entry in sorted_logs.loc[sorted_logs.Shift_id==shift].iterrows():
        assert guard_id == entry.Guard_id
        if entry.Events == "falls asleep":
            start_sleep = entry.Times
        if entry.Events == "wakes up":
            start_hour, start_mins = start_sleep.split(':')
            end_hour, end_mins = entry.Times.split(':')
            flatten_start = 0
            flatten_end = 0
            if start_hour == "00":
                flatten_start += 60
            flatten_start += int(start_mins)
            if end_hour == "00":
                flatten_end += 60
            flatten_end += int(end_mins)
            timesheet[timesheet_idx, flatten_start:flatten_end] += 1

In [24]:
total_sleep_time = np.sum(timesheet, 1)
sleepiest_guard_index = np.argmax(total_sleep_time)

In [25]:
sleepiest_guard = int(guard_id_dict[sleepiest_guard_index])
sleepiest_guard

2917

In [26]:
sleepiest_time = np.argmax(timesheet[sleepiest_guard_index, :]) - 60
sleepiest_time

25

In [27]:
print (sleepiest_time*sleepiest_guard)

72925


# Part Two

In [40]:
sleepiest_guard_index, sleepiest_time_position = np.where(timesheet == np.max(timesheet))

In [42]:
sleepiest_guard = int(guard_id_dict[sleepiest_guard_index[0]])
sleepiest_time = sleepiest_time_position[0] - 60

In [43]:
sleepiest_guard

1489

In [44]:
print (sleepiest_time*sleepiest_guard)

49137
